## Define Operator Algebra

In [1]:
using LinearAlgebra

struct Operator  # Linear Matrix Operators from Matrices to Matrices (and the operator adjoint)
    op
    adj
    sym
end

## Operators
ℒ(A::Matrix)  = Operator(X->A*X   , X->A'*X, "ℒ$(size(A))"  )   # left multiply by A (X → AX)
ℛ(A::Matrix)  = Operator(X->X*A   , X->X*A', "ℛ$(size(A))")     # right multiply by A (X → XA)
ℋ(A::Matrix)  = Operator(X->X.*A  , X->X.*A, "ℋ$(size(A))")    # Hadamard product (elementwise product)
ℐ()  =          Operator(X->X      ,    X->X,    "I")     # identity operator
𝒪()  =          Operator(X->zero(X) , X->zero(X),"𝒪")# zero operator

import Base:  zero, show, adjoint, *, \, ∘, +, -
show(io::IO, M::Operator) = print(io, M.sym)  # pretty printing
zero(::Any) = 𝒪() # Let's make any undefined zero the 𝒪 operator
adjoint(A::Operator) = Operator(A.adj, A.op,  "("*A.sym*")'")
adjoint(B::Bidiagonal) = Bidiagonal(adjoint.(B.dv),adjoint.(B.ev),(B.uplo == 'U') ? :L : :U) # lower to upper
-(A::Operator) = Operator(X->-A.op(X), X->-A.adj(X),"-"*A.sym)
-(::typeof(𝒪), X::Matrix) = -X # 𝒪 - X should be -X
*(A::Operator, X::Matrix) = A.op(X)
\(ℐ::typeof(ℐ()), A::Matrix) = A
∘(A::Operator, B::Operator) = Operator(A.op ∘ B.op, B.adj ∘ A.adj, A.sym*"∘"*B.sym)
# We need [A;B]*C to somehow magically be [AC;BC]
*(M::Adjoint{Operator, Matrix{Operator}},v::Array) = M .* [v]
+(A::Array,x::Number)=A.+x

+ (generic function with 209 methods)

## Example

In [2]:
# Basic Test
B = [ 1 2; 3 4]
M = [10 1;1 10]
C = [ 2 5;4 6]
ℒ(M)

ℒ(2, 2)

In [3]:
ℒ(M) * [ 1 0 ;0 1]

2×2 Matrix{Int64}:
 10   1
  1  10

In [4]:
ℒ(M) * B 

2×2 Matrix{Int64}:
 13  24
 31  42

In [5]:
ℛ(M) * B 

2×2 Matrix{Int64}:
 12  21
 34  43

In [6]:
ℋ(M) * B

2×2 Matrix{Int64}:
 10   2
  3  40

In [7]:
tr( B'*(ℒ(M)*C) ), tr( (ℒ(M)'*B) *C)    # <B,ℒC>=<ℒ'B,C>

(522, 529)

In [8]:
B = [ 1 2; 3 4]
M = Bidiagonal( [ℐ(),ℐ(),ℐ()] , [ℒ(B),ℒ(B)], :L)
display(Matrix(M))

3×3 Matrix{Operator}:
 I        𝒪        𝒪
 ℒ(2, 2)  I        𝒪
 𝒪        ℒ(2, 2)  I

In [9]:
display(Matrix(M'))

3×3 Matrix{Operator}:
 (I)'  (ℒ(2, 2))'  𝒪
 𝒪     (I)'        (ℒ(2, 2))'
 𝒪     𝒪           (I)'

In [10]:
b = [ rand(2,2) for i=1:3]
x = M'\b
display(M'*x .- b)

3-element Vector{Matrix{Float64}}:
 [2.220446049250313e-16 -1.1102230246251565e-16; -2.220446049250313e-16 -1.1102230246251565e-16]
 [-1.1102230246251565e-16 0.0; 0.0 0.0]
 [0.0 0.0; 0.0 0.0]

In [11]:
M = Bidiagonal( [ℐ(),ℐ(),ℐ()] , [ℒ(B),ℒ(B)], :L)
display(Matrix(M))

b = [ rand(2,2) for i=1:3]
display(b)
x = M'\b
display(M'*x .- b)
display(Matrix(M'))

x = M\b
M*x .- b

3×3 Matrix{Operator}:
 I        𝒪        𝒪
 ℒ(2, 2)  I        𝒪
 𝒪        ℒ(2, 2)  I

3-element Vector{Matrix{Float64}}:
 [0.3836980979766623 0.28498631293427434; 0.11411628417553499 0.34083787078443717]
 [0.17317164441252542 0.7434023531071362; 0.8694210590851846 0.8768590876207298]
 [0.5583009788224922 0.6969418939031924; 0.8987263564094704 0.7805917898380544]

3-element Vector{Matrix{Float64}}:
 [-1.1102230246251565e-16 -3.3306690738754696e-16; -1.1102230246251565e-15 -1.1102230246251565e-16]
 [1.1102230246251565e-16 -1.1102230246251565e-16; 1.1102230246251565e-16 0.0]
 [0.0 0.0; 0.0 0.0]

3×3 Matrix{Operator}:
 (I)'  (ℒ(2, 2))'  𝒪
 𝒪     (I)'        (ℒ(2, 2))'
 𝒪     𝒪           (I)'

3-element Vector{Matrix{Float64}}:
 [0.0 0.0; 0.0 0.0]
 [0.0 0.0; 0.0 0.0]
 [-2.220446049250313e-16 2.220446049250313e-16; 3.3306690738754696e-16 3.3306690738754696e-16]

## Simple neural net

In [12]:
using OffsetArrays

h(x) =   exp(-x) # sample activation function
h′(x) = -exp(-x)

function neural_net(params,X₀;h=h,h′= h′)
    T = Matrix{Float64}
    N = length(params)
    X = OffsetArray(Vector{T}(undef,N+1),0:N)   
    Δ = Vector{T}(undef, N)
    X[0] = X₀
    W = first.(params)
    B = last.(params)
    
    for i=1:N         
          X[i] =  h.(W[i]*X[i-1] .+ B[i])
          Δ[i] =  h′.(W[i]*X[i-1] .+ B[i])        
    end 
    X,Δ
end

neural_net (generic function with 1 method)

## Initialization

In [13]:
n = [5,4,3,1]  ## this contains [n₀...n_N]
k = 10 # batchsize
N = length(n)-1 #should be positive
init(sizes...) = 0.01randn(sizes...)
Ws_and_bs =[ [init(n[i+1],n[i]) , init(n[i+1])]  for i=1:N] # The second part of the pair is a vector here
X₀ = init(n[1],k)
y  =  init(n[end],k); #  y is what we will compare X_N against
X,δ = neural_net(Ws_and_bs,X₀) # This has all the X's and δ's

𝓁(x,y) = sum(abs2,x-y)/2 #loss
𝓁′(x,y) = x.-y;

X,δ = neural_net(Ws_and_bs,X₀) # Run the neural net

([[-0.0013288446490908037 -0.018245244648934882 … 0.010365643342355875 -0.010041176535867167; 0.0022076594193477118 0.0023302984846954467 … -0.005459392208937755 -0.009653061180875725; … ; 0.014408997347065607 0.002625646801669439 … -0.016493978540956165 -0.01523029742813828; -0.0004872220267880179 0.01616374006124064 … -0.004204790679877666 0.006482416005138892], [1.0033067333475925 1.0031891575761056 … 1.0034469540584792 1.0033820072187707; 1.0000890847792285 0.9996677638929194 … 0.9992207008362067 0.9993745214014556; 0.9961137252306237 0.995805468002738 … 0.9955544123928837 0.9955757798642912; 1.0174948380173312 1.017321545452243 … 1.0178956096246445 1.017730336832039], [0.9895286091115105 0.9895272674758858 … 0.9895123306580378 0.9895175107771471; 0.960310292594243 0.9603212112921704 … 0.960326071615635 0.9603247105083558; 0.9980598732066502 0.9980556147509339 … 0.9980374631846645 0.9980428997357041], [1.0281548646302205 1.0281547955773334 … 1.028154334306269 1.0281544822982094]], 

In [14]:
# params: `W_i` and `b_i`s: x_{i+1} <- Wi*x_i .+ b_i
Ws_and_bs =[ [init(n[i+1],n[i]) , init(n[i+1],k)]  for i=1:N] # The second part of the pair is a vector here
X₀ = init(n[1],k)
y  =  init(n[end],k); #  y is what we will compare X_N against
Ws_and_bs

3-element Vector{Vector{Matrix{Float64}}}:
 [[-0.012143092646801593 0.003085823585681731 … -0.003167963113614788 0.009596009661363319; -0.004337762112176135 0.01036608887453694 … 0.005581008479239419 -0.0007020755509605791; 0.014853864599875201 0.021559341643004108 … -0.008651655365812966 0.006115819359156278; -0.026280537238715197 0.0071992297108035485 … 0.006212874031915643 0.003297947312389446], [0.010124943618193329 -0.00025042501816379065 … 0.00863297763320263 -0.004354179124755006; -0.001075709695366005 0.0051588316697859715 … -0.006054310659378832 0.011078063256761242; -0.01909868044021105 -0.005730351022052706 … -0.015882019724896477 -0.00048084291354487694; 0.0049544571916539226 -0.0023089690395337783 … -0.0027878323531061178 0.0025997643434722373]]
 [[0.003001198161846007 0.008244343773183107 0.014301576004232304 -0.0005840931884569898; -0.021669175196715865 0.004129064636437034 -0.012704334302578233 -0.002263452940631432; -0.010408180508868421 0.016616016983090703 0.01094143

## Backward diff a neural net with operators

In [15]:
X,δ = neural_net(Ws_and_bs,X₀) # This has all the X's and δ's

## The diagonal matrix
M = Diagonal([ [ℋ(δ[i]) ∘ ℛ(X[i-1])  ℋ(δ[i])] for i=1:N])

## The lower triangular matrix (I-L)
ImL = Bidiagonal([ℐ() for i in 1:N], -[ℋ(δ[i]) ∘ ℒ(Ws_and_bs[i][1]) for i=2:N] , :L)

## gradient of the loss function
g = [ fill(𝒪,N-1) ; [𝓁′(X[N],y)] ] 
∇J = M' * (ImL' \ g)

3-element Vector{Matrix{Matrix{Float64}}}:
 [[-6.385470691240506e-6 -1.3481496004603963e-6 … -1.5878810695035885e-5 -1.4816075665073404e-5; 3.5708348843920667e-6 6.505383638369971e-7 … 8.777301350999524e-6 8.042025689156865e-6; -1.0398290917512726e-6 -1.6016504575810593e-7 … -2.2261982874820615e-6 -1.976400564119576e-6; -5.607325372731401e-7 -1.0767864366481515e-7 … -1.3581750522014745e-6 -1.2589959931849711e-6]; [0.000510399396727627 0.0005316220571892187 … 0.0005288901591086085 0.0005181176123132388; -0.0002809993000758474 -0.00028728428373578786 … -0.00028858484301765274 -0.00027692069873127564; 7.422618881896844e-5 7.618698777236433e-5 … 8.104095540906386e-5 7.39251941667049e-5; 4.451549193646179e-5 4.630286908066782e-5 … 4.680817817913958e-5 4.4710365776625626e-5];;]
 [[0.0760436893216154 0.07642792837507782 0.07648427657059649 0.07614417143430822; 0.2139709907056193 0.21505681819898936 0.21521862645316592 0.21426111731017008; 0.08026338180156754 0.08066607784495629 0.080728129878

In [20]:
#∇Jfd is the gradient calculated with finite differences method
∇Jfd = Ws_and_bs*0
ϵ = Ws_and_bs*0
𝜀 = .0001
for i=1:length(Ws_and_bs), wb=1:2
    for j=1:length(ϵ[i][wb])
        ϵ[i][wb][j] = 𝜀
        ∇Jfd[i][wb][j] = (𝓁(neural_net(Ws_and_bs+ϵ,X₀)[1][N],y).-𝓁(neural_net(Ws_and_bs-ϵ,X₀)[1][N],y))/2𝜀
        ϵ[i][wb][j] = .0
  end
 end

In [21]:
flatten(J) = vcat((x->x[:]).(vcat(J...))...)

flatten (generic function with 1 method)

In [22]:
norm(flatten(∇J)-flatten(∇Jfd))

1.0741248302637355e-7